In [2]:
import pandas as pd

In [3]:
books = pd.read_csv('books_data_classified.csv')

In [5]:
from transformers import pipeline
classifier = pipeline("text-classification", 
                    model="j-hartmann/emotion-english-distilroberta-base",
                    top_k=None)
classifier("I love this!")


Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528691716492176},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419791977852583},
  {'label': 'sadness', 'score': 0.002092393347993493},
  {'label': 'disgust', 'score': 0.001611992483958602},
  {'label': 'fear', 'score': 0.0004138525982853025}]]

In [6]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [9]:
pred = classifier(books["description"][0].split("."))


In [10]:
import numpy as np

In [18]:
emotion_list = ["anger", "disgust", "fear", "joy", "neutral", "sadness", "surprise"]
isbn=[]
emotion_scores={emotion: [] for emotion in emotion_list}    

def get_emotion_scores(predictions):
    per_emotion = {emotion: [] for emotion in emotion_list}
    for pred in predictions:
        sorted_pred = sorted(pred, key=lambda x: x['label'])
        for idx , label in enumerate(emotion_list):
            per_emotion[label].append(sorted_pred[idx]['score'])
    return {emotion : np.max(scores) for emotion, scores in per_emotion.items()}


In [19]:
from tqdm import tqdm

In [20]:
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    pred = classifier(books["description"][i].split("."))
    max_scores = get_emotion_scores(pred)
    for emotion in emotion_list:
        emotion_scores[emotion].append(max_scores[emotion])

100%|██████████| 5197/5197 [07:56<00:00, 10.91it/s]


In [21]:
emotion_df = pd.DataFrame(emotion_scores)
emotion_df["isbn13"] = isbn

In [22]:
emotion_df.head()

,anger,disgust,fear,joy,neutral,sadness,surprise,isbn13
0,0.064134,0.273591,0.928168,0.932798,0.646216,0.967158,0.729602,9780002005883
1,0.612619,0.348284,0.942528,0.704422,0.887940,0.111690,0.252546,9780002261982
2,0.064134,0.104007,0.972321,0.767239,0.549477,0.111690,0.078765,9780006178736
3,0.351485,0.150722,0.360705,0.251882,0.732685,0.111690,0.078765,9780006280897
4,0.081412,0.184496,0.095043,0.040564,0.884390,0.475880,0.078765,9780006280934


In [23]:
books = books.merge(emotion_df, on="isbn13", how="left")

In [29]:
books.head(2)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_subtitle,tag_description,simple_categories,anger,disgust,fear,joy,neutral,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.064134,0.273591,0.928168,0.932798,0.646216,0.967158,0.729602
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612619,0.348284,0.942528,0.704422,0.887940,0.111690,0.252546


In [25]:
books.describe()

,isbn13,published_year,average_rating,num_pages,ratings_count,anger,disgust,fear,joy,neutral,sadness,surprise
count,5.197000e+03,5197.000000,5197.000000,5197.000000,5.197000e+03,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000,5197.000000
mean,9.780667e+12,1999.804118,3.922246,348.472195,2.113112e+04,0.164808,0.200597,0.308601,0.280208,0.760011,0.223608,0.174044
std,5.951053e+08,9.082979,0.324975,229.891672,1.446480e+05,0.218574,0.212761,0.342392,0.317908,0.204867,0.248027,0.189109
min,9.780002e+12,1876.000000,0.000000,0.000000,0.000000e+00,0.000606,0.000821,0.000442,0.000550,0.000981,0.001251,0.000779
25%,9.780313e+12,1998.000000,3.750000,213.000000,1.830000e+02,0.064134,0.104007,0.051363,0.040564,0.549477,0.111690,0.078765
50%,9.780521e+12,2002.000000,3.940000,312.000000,1.125000e+03,0.064134,0.104007,0.093588,0.087731,0.838376,0.111690,0.078765
75%,9.780807e+12,2005.000000,4.120000,416.000000,6.574000e+03,0.138384,0.187477,0.580464,0.498713,0.936846,0.177616,0.198874
max,9.789028e+12,2019.000000,5.000000,3342.000000,5.629932e+06,0.989582,0.989417,0.995326,0.992068,0.974344,0.989361,0.983455


In [26]:
books.to_csv('books_data_classified_with_emotions.csv', index=False)